# Test du Module EPR Plot

Ce notebook teste le nouveau module `eprplot` qui fournit des fonctions simples pour tracer les données EPR obtenues avec `eprload()`.

## Fonctions disponibles:
- `epyr.plot_1d()` - Tracer les spectres EPR 1D
- `epyr.plot_2d_map()` - Tracer les données 2D sous forme de carte colorée
- `epyr.plot_2d_waterfall()` - Tracer les données 2D sous forme de cascade


In [ ]:
# DIAGNOSTIC - Exécutez cette cellule en premier
import sys
print("Python executable:", sys.executable)
print("Python path (premiers 3):")
for p in sys.path[:3]:
    print(" ", p)

# Nettoyage complet des modules
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('epyr')]
for module in modules_to_remove:
    print(f"Suppression du cache: {module}")
    del sys.modules[module]

# Import frais
try:
    import epyr
    print(f"\n✅ EPyR v{epyr.__version__} importé")
    print(f"   Localisation: {epyr.__file__}")
    
    # Test des fonctions
    functions = ['plot_1d', 'plot_2d_map', 'plot_2d_waterfall']
    for func in functions:
        status = "✅" if hasattr(epyr, func) else "❌"
        print(f"   {func}: {status}")
    
    # Si ça ne marche pas, import explicite
    if not hasattr(epyr, 'plot_1d'):
        print("⚠️ Import explicite nécessaire...")
        from epyr.eprplot import plot_1d, plot_2d_map, plot_2d_waterfall
        # Ajouter manuellement au namespace
        epyr.plot_1d = plot_1d
        epyr.plot_2d_map = plot_2d_map  
        epyr.plot_2d_waterfall = plot_2d_waterfall
        print("✅ Fonctions ajoutées manuellement")
        
except Exception as e:
    print(f"❌ Erreur: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Imports pour le reste du notebook
import numpy as np
import matplotlib.pyplot as plt

print("✅ Imports NumPy et Matplotlib réussis")
print("✅ Fonctions EPR plot disponibles - vous pouvez continuer!")

## 1. Test avec données synthétiques 1D

Créons un spectre EPR 1D synthétique pour tester la fonction `plot_1d`.

In [ ]:
# Création de données 1D synthétiques
print("🔬 Génération de données EPR 1D synthétiques...")

# Axe du champ magnétique
field_1d = np.linspace(3300, 3400, 1000)  # Champ en Gauss

# Signal EPR (raie Lorentzienne avec bruit)
center = 3350  # Centre de la raie
linewidth = 8  # Largeur de raie
amplitude = 100

# Raie Lorentzienne
signal_1d = amplitude * linewidth**2 / ((field_1d - center)**2 + linewidth**2)

# Ajout de bruit
noise = np.random.normal(0, 2, len(field_1d))
signal_1d += noise

# Paramètres comme retournés par eprload
params_1d = {
    'XAXIS_NAME': 'Champ Magnétique',
    'XAXIS_UNIT': 'G',
    'MWFQ': 9.5e9  # 9.5 GHz
}

print(f"✅ Données 1D générées: {len(signal_1d)} points")
print(f"📏 Gamme de champ: {field_1d.min():.0f} - {field_1d.max():.0f} G")

In [ ]:
# Test de plot_1d
print("📊 Test de epyr.plot_1d()...")

fig, ax = epyr.plot_1d(
    field_1d, 
    signal_1d, 
    params_1d, 
    title="Spectre EPR 1D Synthétique"
)

plt.show()
print("✅ Tracé 1D réussi!")

## 2. Test avec données complexes 1D

Testons avec des données complexes (comme en EPR pulsée).

In [ ]:
# Données complexes 1D
print("🔬 Génération de données complexes 1D...")

# Signal complexe (partie réelle + imaginaire)
real_part = amplitude * linewidth**2 / ((field_1d - center)**2 + linewidth**2)
imag_part = amplitude * linewidth * (field_1d - center) / ((field_1d - center)**2 + linewidth**2)

complex_signal_1d = real_part + 1j * imag_part
# Ajout de bruit complexe
complex_noise = np.random.normal(0, 1, len(field_1d)) + 1j * np.random.normal(0, 1, len(field_1d))
complex_signal_1d += 0.5 * complex_noise

print(f"✅ Données complexes générées: {complex_signal_1d.dtype}")

In [ ]:
# Test avec données complexes
print("📊 Test de epyr.plot_1d() avec données complexes...")

fig, ax = epyr.plot_1d(
    field_1d,
    complex_signal_1d,
    params_1d,
    title="Spectre EPR 1D Complexe"
)

plt.show()
print("✅ Tracé de données complexes réussi!")

## 3. Test avec données synthétiques 2D

Créons des données 2D pour tester les fonctions `plot_2d_map` et `plot_2d_waterfall`.

In [ ]:
# Création de données 2D synthétiques (balayage angulaire)
print("🔬 Génération de données EPR 2D synthétiques...")

# Axes
field_2d = np.linspace(3300, 3400, 200)  # Champ magnétique
angle_2d = np.linspace(0, 180, 30)       # Angle de rotation

# Matrice 2D
signal_2d = np.zeros((len(angle_2d), len(field_2d)))

# Création d'un signal avec dépendance angulaire
for i, angle in enumerate(angle_2d):
    # Facteur g dépendant de l'angle
    g_factor = 2.0 + 0.1 * np.cos(np.radians(angle))
    resonance_field = 3350 / g_factor * 2.0  # Position de résonance
    linewidth = 10 + 3 * np.sin(np.radians(angle))  # Largeur dépendante de l'angle
    
    # Signal Lorentzien
    signal = 100 * linewidth**2 / ((field_2d - resonance_field)**2 + linewidth**2)
    
    # Ajout de bruit
    noise = np.random.normal(0, 2, len(field_2d))
    signal_2d[i, :] = signal + noise

# Structure de données comme retournée par eprload
x_2d = [field_2d, angle_2d]  # Liste des axes
params_2d = {
    'XAXIS_NAME': 'Champ Magnétique',
    'YAXIS_NAME': 'Angle de Rotation',
    'XAXIS_UNIT': 'G',
    'YAXIS_UNIT': 'deg',
    'MWFQ': 9.5e9
}

print(f"✅ Données 2D générées: {signal_2d.shape}")
print(f"📏 Champ: {field_2d.min():.0f} - {field_2d.max():.0f} G")
print(f"🔄 Angle: {angle_2d.min():.0f} - {angle_2d.max():.0f} deg")

In [ ]:
# Test de plot_2d_map
print("🗺️ Test de epyr.plot_2d_map()...")

fig, ax = epyr.plot_2d_map(
    x_2d,
    signal_2d,
    params_2d,
    title="Carte 2D - Balayage Angulaire EPR",
    cmap="plasma"
)

plt.show()
print("✅ Carte 2D tracée avec succès!")

In [ ]:
# Test de plot_2d_waterfall
print("🌊 Test de epyr.plot_2d_waterfall()...")

fig, ax = epyr.plot_2d_waterfall(
    x_2d,
    signal_2d,
    params_2d,
    title="Tracé en Cascade - Balayage Angulaire EPR",
    offset_factor=0.4
)

plt.show()
print("✅ Tracé en cascade réussi!")

## 4. Test avec données EPR réelles

Testons avec de vraies données EPR si elles sont disponibles.

In [ ]:
# Test avec données réelles
print("📂 Test avec données EPR réelles...")

# Liste des fichiers de test
test_files = [
    "../data/Rabi2D_GdCaWO4_13dB_3057G.DSC",
    "../data/Rabi2D_GdCaWO4_6dB_3770G_2.DSC", 
    "../data/2014_03_19_MgO_300K_111_fullrotation33dB.par"
]

real_data_found = False

for file_path in test_files:
    try:
        print(f"\n📁 Tentative de chargement: {file_path}")
        
        # Charger les données
        x, y, params, filepath = epyr.eprload(file_path, plot_if_possible=False)
        
        if y is not None:
            print(f"✅ Données chargées: {y.shape}")
            print(f"📊 Type: {'Complexe' if np.iscomplexobj(y) else 'Réel'}")
            
            if y.ndim == 1:
                print("📊 Tracé des données 1D...")
                fig, ax = epyr.plot_1d(x, y, params, title=f"Données 1D: {filepath.split('/')[-1]}")
                plt.show()
                
            elif y.ndim == 2:
                print("🗺️ Tracé de la carte 2D...")
                fig1, ax1 = epyr.plot_2d_map(
                    x, y, params, 
                    title=f"Carte 2D: {filepath.split('/')[-1]}"
                )
                plt.show()
                
                print("🌊 Tracé en cascade 2D...")
                fig2, ax2 = epyr.plot_2d_waterfall(
                    x, y, params,
                    title=f"Cascade 2D: {filepath.split('/')[-1]}",
                    max_traces=25  # Limiter pour la performance
                )
                plt.show()
            
            real_data_found = True
            break  # Arrêter après le premier fichier trouvé
        else:
            print(f"❌ Impossible de charger {file_path}")
            
    except Exception as e:
        print(f"❌ Erreur avec {file_path}: {e}")

if not real_data_found:
    print("⚠️ Aucune donnée EPR réelle trouvée. Les tests synthétiques sont suffisants.")

## 5. Test de robustesse

Testons la robustesse des fonctions avec des cas limites.

In [ ]:
# Test avec axes manquants
print("🧪 Test de robustesse - axes manquants...")

# Test 1D sans axe x
try:
    fig, ax = epyr.plot_1d(None, signal_1d, params_1d, title="1D sans axe X")
    plt.show()
    print("✅ Gestion des axes manquants OK")
except Exception as e:
    print(f"❌ Erreur: {e}")

# Test 2D sans paramètres
try:
    fig, ax = epyr.plot_2d_map([field_2d], signal_2d, None, title="2D sans paramètres")
    plt.show()
    print("✅ Gestion des paramètres manquants OK")
except Exception as e:
    print(f"❌ Erreur: {e}")

print("\n🎉 Tests de robustesse terminés!")

## Résumé

Ce notebook a testé avec succès le nouveau module `eprplot` avec:

✅ **Données 1D** (réelles et complexes)  
✅ **Données 2D** (carte colorée et cascade)  
✅ **Données EPR réelles** (si disponibles)  
✅ **Tests de robustesse** (gestion des cas limites)  

### Usage simple:

```python
import epyr

# Charger des données
x, y, params, filepath = epyr.eprload("votre_fichier.dsc")

# Tracer selon le type de données
if y.ndim == 1:
    epyr.plot_1d(x, y, params)
elif y.ndim == 2:
    epyr.plot_2d_map(x, y, params)
    epyr.plot_2d_waterfall(x, y, params)
```

Le module `eprplot` est maintenant prêt pour utilisation dans vos analyses EPR! 🎉